## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

--2021-11-27 11:29:54--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Risoluzione di www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6025:18::a27d:4512
Connessione a www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 301 Moved Permanently
Posizione: /s/raw/xvjzaxzz3ysphme/data_000637.txt [segue]
--2021-11-27 11:29:58--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Riutilizzo della connessione esistente a www.dropbox.com:443.
Richiesta HTTP inviata, in attesa di risposta... 302 Found
Posizione: https://ucca85e7943e4182805224e84812.dl.dropboxusercontent.com/cd/0/inline/BaysJqMFbnwTjhdAP6OkkQQaLt-ALN4jWyinNyW6R9SzWUuhw-RoL4lug5ikmKkYgTNLk53vyqu6ZLCRbUXk8r2y0md3p82Fx_Jd9P5dwp0Y2_z4RQYkXqJ-EHQwUHKW9o70q_oPRa5VUSalPJIBuSTG/file# [segue]
--2021-11-27 11:29:58--  https://ucca85e7943e4182805224e84812.dl.dropboxusercontent.com/cd/0/inline/BaysJqMFbnwTjhdAP6OkkQQaLt-ALN4jWyinNyW6R9SzWUuhw-Ro

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [40]:
import pandas as pd
import numpy as np
import csv
import numpy.random as npr
npr.seed(205678)

N = 200000
dat = pd.DataFrame(pd.read_csv("data_000637.txt", nrows = N))
dat

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
199995,1,0,139,3869202157,76,0
199996,1,0,62,3869202157,76,2
199997,1,1,2,3869202157,94,15
199998,1,1,1,3869202157,96,13


In [41]:
x = max(dat['BX_COUNTER']) - min(dat['BX_COUNTER']) + 1
print("1 ORBIT_CNT equals to {}ns".format(x*25))

1 ORBIT_CNT equals to 89100ns


In [42]:
dat = pd.DataFrame(pd.read_csv("data_000637.txt"))
N = dat.index.stop

time0 = dat['ORBIT_CNT'][0]*89100 + dat['BX_COUNTER'][0]*25 + dat['TDC_MEAS'][0]*(25/30)
time1 = dat['ORBIT_CNT'][N-1]*89100 + dat['BX_COUNTER'][N-1]*25 + dat['TDC_MEAS'][N-1]*(25/30)

time = time1 - time0
print("Data taking lasted {}s".format(time * pow(10,-9)))

Data taking lasted 0.9804160933125s


In [43]:
dat["TIME"] = (dat['ORBIT_CNT']*89100 + dat['BX_COUNTER']*25 + dat['TDC_MEAS']*(25/30))
dat["TIME"] = (dat["TIME"] - dat["TIME"][0])
dat

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME
0,1,0,123,3869200167,2374,26,0.000000e+00
1,1,0,124,3869200167,2374,27,8.125000e-01
2,1,0,63,3869200167,2553,28,4.476625e+03
3,1,0,64,3869200167,2558,19,4.594125e+03
4,1,0,64,3869200167,2760,25,9.649125e+03
...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,9.804161e+08
1310716,1,1,4,3869211171,763,11,9.804161e+08
1310717,1,0,64,3869211171,764,0,9.804161e+08
1310718,1,0,139,3869211171,769,0,9.804163e+08
